In [1]:
import os
import json
import pandas as pd
import traceback

In [7]:
from langchain_openai import ChatOpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
KEY=os.getenv("OPENAI_API_KEY")

In [5]:
print(KEY)

sk-proj-HhoRTlDsrkzVeS346B6oT3BlbkFJ58A2xd6rVgyBLCqlFGM3


In [8]:
llm=ChatOpenAI (openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.3)

In [9]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [10]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [11]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [12]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [14]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [15]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [16]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [18]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [19]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [21]:
path = r"D:\GenAi\mcqgen\data.txt"

In [22]:
with open(path,'r') as f:
    text = f.read()

In [23]:
text

'Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contributed to the modern machine learning technologies as well, including logician Walter Pitts and Warren McCulloch, who proposed the early mathematical models of neural networks to come up with algorithms that mirror human thought processes.[13]

In [24]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [25]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [26]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": text,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

d:\GenAi\mcqgen\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contributed to the modern machine learning technologies as well, including logician Walter Pitts and Warren McCulloch, who proposed the early 

In [27]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1967
Prompt Tokens:1244
Completion Tokens:723
Total Cost:0.003312


In [28]:
response

{'text': 'Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contributed to the modern machine learning technologies as well, including logician Walter Pitts and Warren McCulloch, who proposed the early mathematical models of neural networks to come up with algorithms that mirror human thought proce

In [30]:
quiz=response.get("quiz")

In [33]:
quiz=json.loads(quiz)

In [34]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [35]:
quiz_table_data

[{'MCQ': 'Who introduced a theoretical neural structure formed by certain interactions among nerve cells?',
  'Choices': 'a: Arthur Samuel | b: Donald Hebb | c: Walter Pitts | d: Warren McCulloch',
  'Correct': 'b'},
 {'MCQ': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'Choices': 'a: Cybernet | b: Cybertron | c: RoboLearn | d: TechBrain',
  'Correct': 'b'},
 {'MCQ': 'Which book on research into machine learning during the 1960s is mentioned in the text?',
  'Choices': 'a: The Organization of Behavior | b: Learning Machines | c: Teaching Strategies | d: Pattern Recognition',
  'Correct': 'b'},
 {'MCQ': "What was the purpose of the 'goof' button in the Cybertron machine?",
  'Choices': 'a: To turn off the machine | b: To re-evaluate incorrect decisions | c: To analyze sonar signals | d: To recognize patterns',
  'Correct': 'b'},
 {'MCQ': 'In 1981, what did an artificial neural network learn to recognize from a computer t

In [36]:
data = pd.DataFrame(quiz_table_data)

In [37]:
data

,MCQ,Choices,Correct
0,Who introduced a theoretical neural structure ...,a: Arthur Samuel | b: Donald Hebb | c: Walter ...,b
1,What was the name of the experimental 'learnin...,a: Cybernet | b: Cybertron | c: RoboLearn | d:...,b
2,Which book on research into machine learning d...,a: The Organization of Behavior | b: Learning ...,b
3,What was the purpose of the 'goof' button in t...,a: To turn off the machine | b: To re-evaluate...,b
4,"In 1981, what did an artificial neural network...",a: 40 animals | b: 40 colors | c: 40 character...,c


In [38]:
data.to_csv("machinelearning.csv",index=False)